In [5]:
import pandas

In [6]:
import os
import pandas as pd

# Get the parent directory of the current directory
parent_directory = os.path.dirname(os.getcwd())

# Set the data_directory to the 'kaggle' folder in the parent directory
data_directory = os.path.join(parent_directory, "data", "kaggle","raw")

# Create an empty dictionary to store DataFrames
dataframes = {}

# Iterate through all files in the data_directory
for filename in os.listdir(data_directory):
    # Check if the file is a CSV file
    if filename.endswith(".csv"):
        file_path = os.path.join(data_directory, filename)

        # Read the first 20 lines of the CSV file as a DataFrame
        df = pd.read_csv(file_path, nrows=200)

        # Add the DataFrame to the dictionary with the filename as the key
        dataframes[filename] = df

# Print the DataFrames
for filename, df in dataframes.items():
    print(f"Data for {filename}:\n")



Data for credits.csv:

Data for keywords.csv:

Data for links.csv:

Data for links_small.csv:

Data for movies_metadata.csv:

Data for ratings.csv:

Data for ratings_small.csv:



In [7]:
dataframes.keys()

dict_keys(['credits.csv', 'keywords.csv', 'links.csv', 'links_small.csv', 'movies_metadata.csv', 'ratings.csv', 'ratings_small.csv'])

In [8]:
dataframes['movies_metadata.csv']['overview'].iloc[0]

"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences."

In [9]:
#create data.txt
f = open("data.txt", "w")
with open("data.txt", "a") as f:
    for df in dataframes.keys():
        for col in dataframes[df].columns:
            f.write(col + '\n')


In [10]:
dataframes['keywords.csv'].head(4)

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."


In [11]:
dataframes['movies_metadata.csv']['genres'].iloc[0]

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

In [12]:
dataframes['movies_metadata.csv'].columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [44]:
df0=dataframes['movies_metadata.csv'][['original_title','genres','overview']]

In [45]:
df0=df0.set_index('original_title')

In [61]:
type(df0['genres'].iloc[0])

str

In [62]:
import ast

# Create a function to extract the genre names from a list of genres
def extract_genres(genre_list):
    genres = []
    for genre in genre_list:
        genres.append(genre['name'])
    return genres

# Convert the 'genres' string to a list of dictionaries
df0['genres'] = df0['genres'].apply(lambda x: ast.literal_eval(x))

# Extract the genre names from the list of dictionaries
df0['genre_list'] = df0['genres'].apply(lambda x: extract_genres(x))


In [110]:
processed_df = df0[['genre_list', 'overview']]

In [112]:
#remove rows with null values
processed_df = processed_df.dropna()
#remove rows with empty strings
processed_df = processed_df[processed_df['overview'] != '']
#remove rows with no data
processed_df = processed_df[processed_df['overview'] != 'No overview found.']

In [123]:
import os

# Create the 'processed' directory if it doesn't already exist
if not os.path.exists('../data/processed'):
    os.makedirs('../data/processed')

# Save the DataFrame to a CSV file in the 'processed' directory
processed_df.to_csv('../data/processed/processed_data.csv')


processed dataframe is no in data/processed

In [129]:
df=pd.read_csv('../data/processed/processed_data.csv',index_col=0)

In [133]:
test=df

In [127]:
# transform 'genre_liste' in binary encoded columns
df = df.join(df['genre_list'].str.join('|').str.get_dummies())



In [128]:
df

,genre_list,overview,,',",",A,C,D,F,H,...,l,m,n,o,r,s,t,u,v,y
original_title,,,,,,,,,,,,,,,,,,,,,
Toy Story,"['Animation', 'Comedy', 'Family']","Led by Woody, Andy's toys live happily in his ...",1,1,1,1,1,0,1,0,...,1,1,1,1,0,0,1,0,0,1
Jumanji,"['Adventure', 'Fantasy', 'Family']",When siblings Judy and Peter discover an encha...,1,1,1,1,0,0,1,0,...,1,1,1,0,1,1,1,1,1,1
Grumpier Old Men,"['Romance', 'Comedy']",A family wedding reignites the ancient feud be...,1,1,1,0,1,0,0,0,...,0,1,1,1,0,0,0,0,0,1
Waiting to Exhale,"['Comedy', 'Drama', 'Romance']","Cheated on, mistreated and stepped on, the wom...",1,1,1,0,1,1,0,0,...,0,1,1,1,1,0,0,0,0,1
Father of the Bride Part II,['Comedy'],Just when George Banks has recovered from his ...,0,1,0,0,1,0,0,0,...,0,1,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Strange Days,"['Crime', 'Drama', 'Science Fiction', 'Thriller']",Set in the year 1999 during the last days of t...,1,1,1,0,1,1,1,0,...,1,1,1,1,1,0,1,0,0,0
Les parapluies de Cherbourg,"['Drama', 'Romance']",This simple romantic tragedy begins in 1957. G...,1,1,1,0,0,1,0,0,...,0,1,1,1,1,0,0,0,0,0
The Tie That Binds,['Thriller'],John Netherwood and his wife Leann are fugitiv...,0,1,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
